# 🏠 Análisis de precios de Airbnb en Madrid

<div align="center">
  <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/69/Airbnb_Logo_B%C3%A9lo.svg/2560px-Airbnb_Logo_B%C3%A9lo.svg.png" width="300">
</div>

## 📊 Objetivo del proyecto

Este proyecto tiene como objetivo analizar los factores que influyen en los precios de los alojamientos de Airbnb en Madrid, con especial foco en el perfil de propiedades de nuestro cliente:

- **Tipo de propiedad**: Apartamentos
- **Número de habitaciones**: De 1 a 3 habitaciones
- **Rango de precio**: Menor a 200€ por noche

Mediante técnicas de análisis de datos y modelos predictivos, buscaremos entender:

- ¿Qué características de las propiedades tienen mayor impacto en el precio?
- ¿Cómo influye la ubicación en el coste de los alojamientos?
- ¿Existe una correlación entre las valoraciones de los huéspedes y los precios?
- ¿Qué diferencias hay entre las propiedades de los Superhosts y los anfitriones regulares?
- ¿Cuál sería el precio óptimo para las propiedades de nuestro cliente según sus características?

## 🗂️ Datos

El análisis se basa en un conjunto de datos que incluye información sobre más de 21,000 propiedades en Madrid, con detalles sobre:

- **Condiciones**: precio, noches mínimas/máximas, políticas de cancelación
- **Anfitriones**: tiempo de respuesta, tasa de respuesta, estado de Superhost, verificaciones
- **Ubicación**: vecindario, coordenadas, distancia a puntos de interés
- **Propiedades**: tipo de propiedad, tipo de habitación, capacidad, comodidades
- **Reseñas**: puntuaciones, cantidad de reseñas, idiomas de las reseñas

## 🛠️ Metodología

El proyecto seguirá estos pasos:

1. **Exploración y limpieza de datos**: Análisis de valores nulos, duplicados y outliers
2. **Análisis exploratorio**: Distribución de precios, relaciones entre variables
3. **Visualización**: Mapas de calor de precios por zonas, gráficos de correlación
4. **Segmentación**: Análisis específico del segmento de apartamentos de 1-3 habitaciones con precio menor a 200€
5. **Modelado**: Predicción de precios basada en las características de las propiedades
6. **Conclusiones**: Recomendaciones para optimizar el precio de los apartamentos del cliente

## 👨‍💻 Tecnologías utilizadas

- Python (Pandas, NumPy)
- Visualización (Matplotlib, Seaborn, Folium)
- Modelos de Machine Learning (Scikit-learn)
- Jupyter Notebooks

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Configurar el estilo de las visualizaciones
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')

In [17]:
# Directorio donde están los archivos parquet
data_dir = '../data/' 

# Crear un diccionario para almacenar los dataframes
dataframes = {}

# Leer todos los archivos parquet en el directorio
for file in os.listdir(data_dir):
    if file.endswith('.parquet'):
        file_path = os.path.join(data_dir, file)
        # Usar el nombre del archivo (sin la extensión) como clave del diccionario
        df_name = os.path.splitext("df_" + ("".join(list(file.split ("_")[-1]))))[0]
        # Leer el archivo parquet
        dataframes[df_name] = pd.read_parquet(file_path)
        print(f"Archivo {file} leído correctamente")

# Mostrar los nombres de los dataframes disponibles
print("\nDataframes disponibles:")
for name in dataframes.keys():
    print(f"- {name}")

Archivo airbnb_madrid_conditions.parquet leído correctamente
Archivo airbnb_madrid_host.parquet leído correctamente
Archivo airbnb_madrid_location.parquet leído correctamente
Archivo airbnb_madrid_property.parquet leído correctamente
Archivo airbnb_madrid_reviews.parquet leído correctamente

Dataframes disponibles:
- df_conditions
- df_host
- df_location
- df_property
- df_reviews


In [15]:
# Inspección inicial de cada dataframe
print("\n" + "="*50)
print("INSPECCIÓN INICIAL DE DATAFRAMES")
print("="*50)

for name, df in dataframes.items():
    print(f"\n{'-'*40}")
    print(f"Dataframe: {name}")
    print(f"{'-'*40}")
    
    # Forma del dataframe (filas, columnas)
    print(f"Shape: {df.shape} (filas, columnas)")
    
    # Nombres de las columnas
    print(f"\nColumnas: {list(df.columns)}")
    
    # Tipos de datos
    print("\nTipos de datos:")
    print(df.dtypes)
    
    # Valores nulos
    null_count = df.isnull().sum()
    print("\nValores nulos por columna:")
    print(null_count[null_count > 0] if null_count.any() > 0 else "No hay valores nulos")
    
    # Duplicados
    duplicates = df.duplicated().sum()
    print(f"\nFilas duplicadas: {duplicates}")
    
    # Posibles columnas de clave (suponiendo que son columnas con valores únicos)
    unique_cols = []
    for col in df.columns:
        if df[col].nunique() == len(df):
            unique_cols.append(col)
    print("\nPosibles columnas clave (valores únicos):")
    print(unique_cols if unique_cols else "No se encontraron columnas con valores únicos")
    
    # Muestra de los primeros registros
    print("\nPrimeras 3 filas:")
    print(df.head(3))

print("\n" + "="*50)
print("RESUMEN DE POSIBLES CLAVES PARA MERGE")
print("="*50)
for name, df in dataframes.items():
    unique_cols = [col for col in df.columns if df[col].nunique() == len(df)]
    print(f"{name}: {unique_cols}")


INSPECCIÓN INICIAL DE DATAFRAMES

----------------------------------------
Dataframe: df_conditions
----------------------------------------
Shape: (21020, 7) (filas, columnas)

Columnas: ['id', 'price', 'minimum_nights', 'maximum_nights', 'cancellation_policy', 'require_guest_profile_picture', 'require_guest_phone_verification']

Tipos de datos:
id                                    int64
price                               float64
minimum_nights                        int64
maximum_nights                        int64
cancellation_policy                  object
require_guest_profile_picture         int64
require_guest_phone_verification      int64
dtype: object

Valores nulos por columna:
No hay valores nulos

Filas duplicadas: 0

Posibles columnas clave (valores únicos):
['id']

Primeras 3 filas:
   id  price  minimum_nights  maximum_nights          cancellation_policy  \
0   0   70.0               1             365                     flexible   
1   1   17.0               4       

In [18]:
df_conditions = dataframes['df_conditions']
df_host = dataframes['df_host']
df_location = dataframes['df_location']
df_property = dataframes['df_property']
df_reviews = dataframes['df_reviews']

In [20]:
# Merge de todos los dataframes usando la columna 'id'
df_merged = df_conditions.merge(df_host, on='id', how='left')\
                        .merge(df_location, on='id', how='left')\
                        .merge(df_property, on='id', how='left')\
                        .merge(df_reviews, on='id', how='left')

# Verificamos el shape del dataframe resultante
print(f"Shape del dataframe merged: {df_merged.shape}")

# Verificamos que no hemos perdido filas
print(f"Número de filas original: {len(df_conditions)}")
print(f"Número de filas después del merge: {len(df_merged)}")

# Verificamos que tenemos todas las columnas
print(f"Número total de columnas esperadas: {sum([df.shape[1] for df in dataframes.values()]) - 4*1}")  # Restamos 4 porque 'id' aparece en cada dataframe
print(f"Número de columnas después del merge: {df_merged.shape[1]}")
# Verificamos valores nulos en el dataframe resultante
null_counts = df_merged.isnull().sum()
print("\nValores nulos en el dataframe merged:")
print(null_counts[null_counts > 0])

Shape del dataframe merged: (21020, 52)
Número de filas original: 21020
Número de filas después del merge: 21020
Número total de columnas esperadas: 52
Número de columnas después del merge: 52

Valores nulos en el dataframe merged:
host_response_time             4464
host_response_rate             4464
review_scores_rating           4294
review_scores_accuracy         4295
review_scores_cleanliness      4293
review_scores_checkin          4292
review_scores_communication    4292
review_scores_location         4295
review_scores_value            4296
reviews_per_month              4038
number_of_reviews_en           4038
number_of_reviews_es           4038
number_of_reviews_otros        4038
dtype: int64
